# Memory: A Deep Dive: Practical Implementation (Self Healing Codebase)


## Overview
This code implements a workflow-based error detection and correction system that combines LangGraph and LLM capabilities to detect runtime errors, generate fixes, and maintain a memory of bug patterns. The system takes function definitions and runtime arguments, processes them through a graph-based workflow, and maintains a hierarchical error management system (error → analysis → fix → validation).

## Motivation
Several key factors motivate this implementation:

1. **Automated Error Resolution**
   - Manual debugging is time-consuming and error-prone
   - Automated fix generation streamlines the correction process
   - LLMs can provide context-aware code repairs

2. **Stateful Error Management**
   - Need for systems that track error correction workflow
   - Importance of maintaining error context
   - Value of tracking fix attempts

3. **Structured Bug Knowledge**
   - Need for consistent error pattern recognition
   - Importance of maintaining fix history
   - Benefits of hierarchical error categorization

4. **Runtime Code Modification**
   - Recognition that code fixes need controlled deployment
   - Value of state tracking during modifications
   - Importance of safe runtime patching

## Key Components
1. **State Management System**: 
   - Maintains workflow state using Pydantic models
   - Tracks function references, errors, and fixes
   - Ensures type safety and execution validation

2. **LLM Integration**: 
   - Leverages LLM for code analysis and generation
   - Produces fixes based on error types:
     - Runtime Errors
     - Logic Errors
     - Type Errors
     - Resource Errors

3. **Graph-based Workflow**: 
   - Uses LangGraph's StateGraph for orchestration
   - Implements error detection nodes
   - Controls fix generation through edges

4. **Memory Structure**:
   - Follows error pattern organization
   - Maintains context with each fix
   - Supports multiple error categories

## Visual Overview
A flowchart representing the design and flow of the workflow.

<div style="max-width:600px;">
    
![image.png](../images/self_healing_code.png)
    
</div>

## Conclusion
This implementation demonstrates a practical approach to automated code healing. The system combines graph-based workflow management with LLM capabilities, allowing for structured error correction while maintaining clear process control.

Key advantages include:
- Automated error detection and correction
- Clear state management
- Pattern-based fix generation
- Safe runtime code modification

Future improvements could focus on:
- Enhanced fix validation
- Multiple fix generation
- State persistence strategies
- Memory optimization techniques

This system provides a foundation for building more sophisticated self-healing systems, particularly in applications requiring runtime error correction and pattern learning.

# Dependencies and Imports
Install dependencies and import libraries.

In [1]:
%%capture

!pip install langgraph
!pip install langgraph-sdk
!pip install langgraph-checkpoint-sqlite
!pip install langchain-community
!pip install langchain-core
!pip install langchain-openai

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

from pydantic import BaseModel
from typing import Optional, Callable

import json
import os
import types
import inspect
import sys


## Clients
Import API keys and instantiate clients.

In [11]:
os.environ['OPENAI_API_KEY'] = 'YOUR-API-KEY'
llm = ChatOpenAI(model='gpt-4o-mini')

## Define Agent State
We'll define the state that our agent will maintain throughout its operation.


In [4]:
class State(BaseModel):
    function: Callable
    function_string: str
    arguments: list
    error: bool
    error_description: str = ''
    new_function_string: str = ''
    bug_report: str = ''
    memories: list = []
    memory_indexes_to_update: list = []


## Define Code Healing Node Functions
Now we'll define the code healing node functions that our agent will use: run_code, update_code and fix_code.


In [5]:
def run_code(state: State):
    ''' Run Arbitrary Code '''
    try:
        result = state.function(*state.arguments)
    except Exception as e:
        state.error = True
        state.error_description = str(e)
    return state


def update_code(state: State):
    ''' Update Arbitratry Code '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with fixing a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}' 
        'You must provide a fix for the present error only.'
        'The bug fix should handle the thrown error case gracefully by returning an error message.'
        'Do not raise an error in your bug fix.'
        'The function must use the exact same name and parameters.'
        'Your response must contain only the function definition with no additional text.'
        'Your response must not contain any additional formatting, such as code delimiters or language declarations.'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    new_function_string = llm.invoke([message]).content.strip()

    state.new_function_string = new_function_string
    return state


def fix_code(state: State):
    ''' Fix Arbitrary Code '''
    try:        
        # Store the new function as a string
        new_code = state.new_function_string
        
        # Create namespace for new function
        namespace = {}
        
        # Execute new code in namespace
        exec(new_code, namespace)
        
        # Get the new function
        new_divide = namespace['divide_two_numbers']
        
        # Update state
        state.function = new_divide
        state.error = False
                
    except Exception as e:
        print(f'Update failed: {e}')
    return state

## Define Bug Reporting Node Functions
Now we'll define the bug reporting node functions that our agent will use: bug_report_node, memory_search_node, memory_generation_node and memory_modification_node.

In [6]:
def bug_report_node(state: State):
    ''' Generate Bug Report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with generating a bug report for a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}'
        'Your response must be a comprehensive string including only crucial information on the bug report'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    bug_report = llm.invoke([message]).content.strip()

    state.bug_report = bug_report
    return state


def memory_search_node(state: State):
    ''' Find memories relevant to the current bug report '''
    prompt = ChatPromptTemplate.from_template(
        'Given the following bug report and existing memories, identify which memories are relevant and need updating:'
        'Bug Report: {bug_report}'
        'Existing Memories: {memories}'
        'The response should be a list of integers representing memory indexes to be updated'
        'Response Format: [memory_index, ... , memory_index]'
        'Your response must not contain any additional text.'
        
    )
    
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
        memories=state.memories
    ))
    
    response = llm.invoke([message]).content.strip()
    memory_indexes_to_update = json.loads(response)
    state.memory_indexes_to_update = memory_indexes_to_update

    return state


def memory_generation_node(state: State):
    ''' Generate relevant memories based on new bug report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with archiving a bug report for a Python function that raised an error.'
        'Bug Report: {bug_report}.'
        'Your response must be a concise string including only crucial information on the bug report for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )
    
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
    ))
    
    response = llm.invoke([message]).content.strip()
    
    state.memories.append(response)
        
    return state


def memory_modification_node(state: State):
    ''' Modify relevant memories based on new interaction '''
    prompt = ChatPromptTemplate.from_template(
        'Update the following memories based on the new interaction:'
        'Bug Report: {bug_report}'
        'Memory to Update: {memory_to_update}'
        'Response Format: # memory_label ## updated_memory_content ### updated_memory_context'
    )
    memory_to_update_index = state.memory_indexes_to_update.pop(0)
    memory_to_update = state.memories[memory_to_update_index]
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
        memory_to_update=memory_to_update,
    ))
    
    response = llm.invoke([message]).content.strip()
    
    state.memories[memory_to_update_index] = response
        
    return state
    

## Define Edge Functions
Now we'll define the conditional edge function that our agent will use to control the workflow.

In [7]:
def error_router(state: State):
    if state.error:
        print('Error in code')
        return 'bug_report_node'
    else:
        print('Code is healthy')
        return END


def modification_router(state: State):
    if state.memory_indexes_to_update:
        return 'memory_modification_node'
    else:
        return 'memory_generation_node'


def update_memory(state: State):
    if state.memory_indexes_to_update:
        return 'memory_modification_node'
    else:
        return 'update_code'

## Build Workflow
Now we'll create our workflow and compile it.


In [8]:
builder = StateGraph(State)

# Add nodes to the graph
builder.add_node('run_code', run_code)
builder.add_node('update_code', update_code)
builder.add_node('fix_code', fix_code)
builder.add_node('bug_report_node', bug_report_node)
builder.add_node('memory_search_node', memory_search_node)
builder.add_node('memory_modification_node', memory_modification_node)
builder.add_node('memory_generation_node', memory_generation_node)


# Add edges to the graph
builder.set_entry_point('run_code')
builder.add_conditional_edges('run_code', error_router)
builder.add_edge('bug_report_node', 'memory_search_node')
builder.add_conditional_edges('memory_search_node', modification_router)
builder.add_edge('memory_generation_node', 'update_code')
builder.add_conditional_edges('memory_modification_node', update_memory)

builder.add_edge('update_code', 'fix_code')
builder.add_edge('fix_code', 'run_code')

# Compile the graph
graph = builder.compile()

# Main Function
Define the function that runs the instanciates the workflow and its state.

In [9]:
def run_self_healing_code_system(function, arguments):

    state = State(
        error=False,
        function=function,
        function_string=inspect.getsource(function),
        arguments=arguments,
    )
    
    for output in graph.stream(state):
        last_state = output
        pass

    return last_state
    


# Run Program
Instanciate the main function and observe outputs.

In [10]:
# Define sample function to heal
def divide_two_numbers(a, b):
    return a/b

run_self_healing_code_system(divide_two_numbers, [10, 0])
run_self_healing_code_system(divide_two_numbers, ['a', 0])

Error in code
Code is healthy
Error in code
Code is healthy


{'run_code': {'function': <function divide_two_numbers(a, b)>,
  'function_string': 'def divide_two_numbers(a, b):\n    return a/b\n',
  'arguments': ['a', 0],
  'error': False,
  'error_description': "unsupported operand type(s) for /: 'str' and 'int'",
  'new_function_string': 'def divide_two_numbers(a, b):\n    if isinstance(a, str) or isinstance(b, str):\n        return "Error: both arguments must be numbers."\n    return a / b',
  'bug_report': '**Bug Report**\n\n**Title:** TypeError in `divide_two_numbers` function\n\n**Function:** `divide_two_numbers(a, b)`\n\n**Error Message:** `unsupported operand type(s) for /: \'str\' and \'int\'`\n\n**Description:** The function raises a TypeError when the first argument `a` is of type `str` and the second argument `b` is of type `int`. The division operation does not support these operand types.\n\n**Steps to Reproduce:**\n1. Call the function with a string as the first argument and an integer as the second argument, e.g., `divide_two_numb